In [8]:
import tensorflow as tf
from tensorflow import keras

((x_train, y_train), (x_test, y_test)) = keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape((60000, 28, 28, 1))
x_test = x_test.reshape((10000, 28, 28, 1))
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

Question 1 : A Convolutional Neural Network 
---

In this question, we ask you to build a model closely matching the above picture.

The model must have:

- **3 convolutional blocks** with a number of filters of respectively $16$, $32$, and $64$; **each block containing**:
    - A *Conv2D* layer with a *ReLU* activation.
    - A Max Pooling layer ($2,2$ pool size)
- A dense layer with $28$ hidden units.
- A dense output layer with a softmax activation.

You are free to play with the meta-parameters of each of these layers, as long as you respect this structure.

Implement you neural network in the variable *model*. Just define and compile the network, don't fit it on the training data (your submission will likely time out if you do!).

In [79]:
kernel_size = (2, 2)
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, kernel_size=kernel_size, activation='relu', input_shape=(28, 28, 1)), 
    tf.keras.layers.MaxPooling2D(pool_size=2), 
    
    tf.keras.layers.Conv2D(32, kernel_size=kernel_size, activation='relu'), 
    tf.keras.layers.MaxPooling2D(pool_size=2), 
    
    tf.keras.layers.Conv2D(64, kernel_size=kernel_size, activation='relu'), 
    tf.keras.layers.MaxPooling2D(pool_size=2), 
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(28, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
    
], name='convolutional_net')

model.compile(
    loss="categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999),
    metrics=["accuracy"]
)

In [81]:
def prepare_dataset(X, y, batch_size, shuffle=True, augment=True):
    augmented_args = dict(
        rotation_range = 8,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        shear_range = 0.3,# shear angle in counter-clockwise direction in degrees  
        width_shift_range=0.08,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.08,  # randomly shift images vertically (fraction of total height)
        vertical_flip=True # randomly flip images
    
        # horizontal_flip=True,
        # vertical_flip=True,
        # rotation_range=20,
        # width_shift_range=0.2,
        # height_shift_range=0.2
        )
    args = {} if not augment else augmented_args
    
    ds = tf.keras.preprocessing.image.ImageDataGenerator(**args).flow(X, y, batch_size=batch_size, shuffle=shuffle)

    return ds

train_ds = prepare_dataset(x_train, y_train, 256, augment=False)

callbacks = [
    # tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(patience=3, verbose=True)
    # tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
]

model.fit(
    train_ds,
    validation_data=(x_test, y_test),
    epochs=30,
    callbacks=callbacks
)

model.save(f"./models/{model.name}.model", save_format='h5')

Epoch 1/30
235/235 [==============================] - 2s 9ms/step - loss: 0.1977 - accuracy: 0.9298 - val_loss: 0.3724 - val_accuracy: 0.8753
Epoch 2/30
235/235 [==============================] - 2s 9ms/step - loss: 0.1975 - accuracy: 0.9302 - val_loss: 0.3725 - val_accuracy: 0.8758
Epoch 3/30
235/235 [==============================] - 2s 9ms/step - loss: 0.1973 - accuracy: 0.9301 - val_loss: 0.3727 - val_accuracy: 0.8761
Epoch 4/30
235/235 [==============================] - 2s 9ms/step - loss: 0.1971 - accuracy: 0.9303 - val_loss: 0.3727 - val_accuracy: 0.8752

Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.
Epoch 5/30
235/235 [==============================] - 2s 9ms/step - loss: 0.1966 - accuracy: 0.9304 - val_loss: 0.3727 - val_accuracy: 0.8758
Epoch 6/30
235/235 [==============================] - 2s 9ms/step - loss: 0.1965 - accuracy: 0.9306 - val_loss: 0.3728 - val_accuracy: 0.8759
Epoch 7/30
235/235 [==============================] - 2s 9ms/step 